In [ ]:
from collections import defaultdict

In [ ]:
from collections import defaultdict

class FPNode:
  def __init__ (self,item,count,parent):
    self.item=item
    self.count=count
    self.parent=parent
    self.children={}
    self.next=None # Add next attribute for header table linking

  def increment(self,count):
    self.count += count # Correct increment logic

  def build_fp_tree(self, transactions,min_support):
    #step 1: Count frequemcy of items
    item_counts=defaultdict(int)
    for transaction in transactions:
      for item in transaction:
        item_counts[item]+=1

    #step 2: remove infrequent items
    item_counts={item:count for item,count in item_counts.items() # Corrected .item() to .items()
    if count>=min_support}
    if not item_counts:
      return None,None

    #step 3: create header table
    header_table={item:[count,None] for item,count in
                  sorted(item_counts.items(), key=lambda item:item[1], reverse=True)} # Sort header table by frequency
    root = FPNode(None,1,None)

    #step 4: insert transactions
    for transaction in transactions:
      transaction=[item for item in transaction if item in item_counts]
      transaction.sort(key=lambda x:header_table[x][0],reverse=True)
      current=root
      for item in transaction:
        if item not in current.children:
          new_node=FPNode(item,1,current)
          current.children[item] = new_node # Add new node to children
          #update header table links
          if header_table[item][1] is None:
            header_table[item][1]=new_node
          else:
            node=header_table[item][1]
            while node.next:
              node=node.next
            node.next=new_node
        else:
          current.children[item].increment(1)
        current=current.children[item]
    return root,header_table

  def ascend_fp_tree(self,node): # Make ascend_fp_tree a method
    path=[]
    while node.parent and node.parent.item:
      node=node.parent
      path.append(node.item)
    return path

  def find_prefix_paths(self,base_pat,node): # Make find_prefix_paths a method and corrected name
    cond_pats=[]
    while node:
      path=self.ascend_fp_tree(node) # Call ascend_fp_tree as a method
      if path:
        cond_pats.append((path,node.count))
      node=node.next # Move to the next node in the linked list
    return cond_pats

  def mine_fp_tree(self,header_table,min_support,prefix,frequent_patterns): # Make mine_fp_tree a method
     items = sorted(header_table.items(), key=lambda item:item[1][0]) # Sort by support
     for item, (support,node) in items:
      new_freq_set=prefix.copy()
      new_freq_set.add(item)
      frequent_patterns[frozenset(new_freq_set)]=support
      cond_paths=self.find_prefix_paths(item,node) # Call find_prefix_paths as a method
      cond_trans=[]
      for path,count in cond_paths:
        cond_trans.extend([path]*count)

      cond_tree,cond_header=self.build_fp_tree(cond_trans,min_support) # Call build_fp_tree as a method
      if cond_tree is not None:
        self.mine_fp_tree(cond_header,min_support,new_freq_set,frequent_patterns) # Call mine_fp_tree as a method

In [ ]:
#example run
transactions=[
    ['bread','milk'],
    ['bread','diaper','beer','egg'],
    ['milk','diaper','beer','cola'],
    ['bread','milk','diaper','beer'],
    ['bread','milk','diaper','cola']
]

min_support=2
fp_tree_builder = FPNode(None, None, None)  # Create an instance of the class
root,header_table = fp_tree_builder.build_fp_tree(transactions,min_support)
frequent_patterns={}
fp_tree_builder.mine_fp_tree(header_table,min_support,set(),frequent_patterns)
for pattern,support in frequent_patterns.items():
  print(list(pattern),'->',support)

['cola'] -> 2
['diaper', 'cola'] -> 2
['milk', 'cola'] -> 2
['milk', 'diaper', 'cola'] -> 2
['beer'] -> 3
['beer', 'bread'] -> 2
['beer', 'diaper', 'bread'] -> 2
['beer', 'milk'] -> 2
['beer', 'milk', 'diaper'] -> 2
['beer', 'diaper'] -> 3
['bread'] -> 4
['milk'] -> 4
['milk', 'bread'] -> 3
['diaper'] -> 4
['diaper', 'bread'] -> 3
['milk', 'diaper', 'bread'] -> 2
['milk', 'diaper'] -> 3
